### Load all required libraries:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!conda install -c conda-forge folium=0.5.0
import folium

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.3.0               |        py36_1001         533 KB  conda-forge
    pandas-0.23.4              |   py36hf8a1672_0        27.8 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        28.4 MB

The following NEW packages will be INSTALLED:

    altair:  2.3.0-py36_1001       conda-forge
    branca:  0.3.1-py_0            conda-forge
    folium:  0.5.0-py_0            conda-forge
    v

### Get the data and transfrom into pandas data drame

Please note i am not using BeautifulSoup as in pandas we already have built in method to get the data from html

In [6]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
link= pd.read_html(wikipedia_link)
link[0].rename(columns=link[0].iloc[0],inplace=True)
df=link[0].drop(0,axis=0,)
df.head()

Postcode           Borough     Neighbourhood
1      M1A      Not assigned      Not assigned
2      M2A      Not assigned      Not assigned
3      M3A        North York         Parkwoods
4      M4A        North York  Victoria Village
5      M5A  Downtown Toronto      Harbourfront

### Clean the data frame

1. Will be removing Non assigned data.
2. If the neighourhood having same post code,Will combine those neighourhood
3. If a cell has a borough but a Not assigned neighborhood, then will make neighborhood as same as borough

In [7]:
j=0
for s,l in zip(df['Neighbourhood'],df['Borough']):
    j+=1
    if s=='Not assigned' and l!='Not assigned':
        df.at[j, 'Neighbourhood']=l    
    else:
        continue
df.replace('Not assigned',np.nan,inplace=True)
df.dropna(subset=['Borough','Neighbourhood'],axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)
dfnew=df.groupby(['Postcode','Borough'],as_index=False,sort=False,).agg(lambda x: ', '.join(set(x.dropna())))
dfnew

Postcode           Borough  \
0        M3A        North York   
1        M4A        North York   
2        M5A  Downtown Toronto   
3        M6A        North York   
4        M7A      Queen's Park   
5        M9A         Etobicoke   
6        M1B       Scarborough   
7        M3B        North York   
8        M4B         East York   
9        M5B  Downtown Toronto   
10       M6B        North York   
11       M9B         Etobicoke   
12       M1C       Scarborough   
13       M3C        North York   
14       M4C         East York   
15       M5C  Downtown Toronto   
16       M6C              York   
17       M9C         Etobicoke   
18       M1E       Scarborough   
19       M4E      East Toronto   
20       M5E  Downtown Toronto   
21       M6E              York   
22       M1G       Scarborough   
23       M4G         East York   
24       M5G  Downtown Toronto   
25       M6G  Downtown Toronto   
26       M1H       Scarborough   
27       M2H        North York   
28       M3H        North York   
29       M4H         East York   
..       ...               ...   
73       M4R   Central Toronto   
74       M5R   Central Toronto   
75       M6R      West Toronto   
76       M7R       Mississauga   
77       M9R         Etobicoke   
78       M1S       Scarborough   
79       M4S   Central Toronto   
80       M5S  Downtown Toronto   
81       M6S      West Toronto   
82       M1T       Scarborough   
83       M4T   Central Toronto   
84       M5T  Downtown Toronto   
85       M1V       Scarborough   
86       M4V   Central Toronto   
87       M5V  Downtown Toronto   
88       M8V         Etobicoke   
89       M9V         Etobicoke   
90       M1W       Scarborough   
91       M4W  Downtown Toronto   
92       M5W  Downtown Toronto   
93       M8W         Etobicoke   
94       M9W         Etobicoke   
95       M1X       Scarborough   
96       M4X  Downtown Toronto   
97       M5X  Downtown Toronto   
98       M8X         Etobicoke   
99       M4Y  Downtown Toronto   
100      M7Y      East Toronto   
101      M8Y         Etobicoke   
102      M8Z         Etobicoke   

                                         Neighbourhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Harbourfront, Regent Park  
3                     Lawrence Manor, Lawrence Heights  
4                                         Queen's Park  
5                                     Islington Avenue  
6                                       Malvern, Rouge  
7                                      Don Mills North  
8                      Woodbine Gardens, Parkview Hill  
9                             Ryerson, Garden District  
10                                           Glencairn  
11   Princess Gardens, West Deane Park, Martin Grov...  
12              Highland Creek, Rouge Hill, Port Union  
13                    Don Mills South, Flemingdon Park  
14                                    Woodbine Heights  
15                                      St. James Town  
16                                  Humewood-Cedarvale  
17   Markland Wood, Old Burnhamthorpe, Bloordale Ga...  
18                   Morningside, Guildwood, West Hill  
19                                         The Beaches  
20                                         Berczy Park  
21                                 Caledonia-Fairbanks  
22                                              Woburn  
23                                             Leaside  
24                                  Central Bay Street  
25                                            Christie  
26                                           Cedarbrae  
27                                   Hillcrest Village  
28     Downsview North, Wilson Heights, Bathurst Manor  
29                                    Thorncliffe Park  
..                                                 ...  
73                                  North Toronto West  
74            

### Assignment 1 : Print DataFrame Shape


In [12]:
dfnew.shape

(103, 3)

### Get the latitude and the longitude coordinates of each neighborhood

In [12]:
link='http://cocl.us/Geospatial_data'
data= pd.read_csv(link)
data.columns = ['Postcode', 'Latitude', 'Longitude']
data.head()

Postcode   Latitude  Longitude
0      M1B  43.806686 -79.194353
1      M1C  43.784535 -79.160497
2      M1E  43.763573 -79.188711
3      M1G  43.770992 -79.216917
4      M1H  43.773136 -79.239476

In [14]:
dfgeo = pd.merge(dfnew, data, on=['Postcode'], how='inner')

### Assignment 2 : Print dfgeo

In [18]:
dfgeo.head(10)

Postcode           Borough                     Neighbourhood   Latitude  \
0      M3A        North York                         Parkwoods  43.753259   
1      M4A        North York                  Victoria Village  43.725882   
2      M5A  Downtown Toronto         Harbourfront, Regent Park  43.654260   
3      M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4      M7A      Queen's Park                      Queen's Park  43.662301   
5      M9A         Etobicoke                  Islington Avenue  43.667856   
6      M1B       Scarborough                    Malvern, Rouge  43.806686   
7      M3B        North York                   Don Mills North  43.745906   
8      M4B         East York   Woodbine Gardens, Parkview Hill  43.706397   
9      M5B  Downtown Toronto          Ryerson, Garden District  43.657162   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494  
5 -79.532242  
6 -79.194353  
7 -79.352188  
8 -79.309937  
9 -79.378937

In [21]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [22]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


### Create map of Toronto  using latitude and longitude values


In [ ]:
map_geo = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(full_table['Latitude'], full_table['Longitude'], full_table['Neighbourhood\n']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_geo)  
    
map_geo